In [114]:
import keras
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from keras.layers import Dense

from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

%matplotlib inline

In [115]:
from numpy import genfromtxt
EEG = genfromtxt("Confusion during MOOC/EEG_data.csv", delimiter=",")

In [116]:
#6의 raw데이터에 이상이 있으므로 6을 제외하고 분석
EEG=EEG[1:,:]
EEG=pd.DataFrame(EEG)

In [117]:
remove_6=EEG[EEG[0]==6].index
print(EEG.shape, remove_6.shape)

(12811, 15) (1275,)


In [118]:
EEG=EEG.drop(remove_6)
print(EEG.shape)

(11536, 15)


In [119]:
c_EEG=EEG[EEG[14]==0].index
nc_EEG=EEG[EEG[14]==1].index

In [165]:
ConfusedEEG=EEG.drop(c_EEG)
NonConfusedEEG=EEG.drop(nc_EEG)

In [166]:
ConfusedEEG=ConfusedEEG.values
NonConfusedEEG=NonConfusedEEG.values

In [167]:
import random
index=[]
for i in range(7) :
    index.append(random.randint(0,5606))
    
NonConfusedEEG=pd.DataFrame(NonConfusedEEG)
NonConfusedEEG=NonConfusedEEG.drop(index)
NonConfusedEEG=NonConfusedEEG.values

In [168]:
index=[]
for i in range(29) :
    index.append(random.randint(0,5928))
    
ConfusedEEG=pd.DataFrame(ConfusedEEG)
ConfusedEEG=ConfusedEEG.drop(index)
ConfusedEEG=ConfusedEEG.values

In [170]:
X=ConfusedEEG
NX=NonConfusedEEG

(5900, 15) (5600, 15)


In [ ]:
#동영상, raw, 주파수 data 사용
X1=pd.DataFrame(X)
X1=X1.drop(X1.columns[[0,2,3,13,14]], axis='columns')
#사람 정보, raw, 주파수 data 사용
X2=pd.DataFrame(X)
X2=X2.drop(X2.columns[[1,2,3,13,14]], axis='columns')
#동영상, raw, 주파수 data 사용
X3=pd.DataFrame(X)
X3=X3.drop(X3.columns[[0,2,3,4,13,14]], axis='columns')
#사람 정보, 주파수 data 사용
X4=pd.DataFrame(X)
X4=X4.drop(X4.columns[[1,2,3,4,13,14]], axis='columns')

In [172]:
#동영상, raw, 주파수 data 사용
NX1=pd.DataFrame(NX)
NX1=NX1.drop(NX1.columns[[0,2,3,13,14]], axis='columns')
#사람 정보, raw, 주파수 data 사용
NX2=pd.DataFrame(NX)
NX2=NX2.drop(NX2.columns[[1,2,3,13,14]], axis='columns')
#동영상, raw, 주파수 data 사용
NX3=pd.DataFrame(NX)
NX3=NX3.drop(NX3.columns[[0,2,3,4,13,14]], axis='columns')
#사람 정보, 주파수 data 사용
NX4=pd.DataFrame(NX)
NX4=NX4.drop(NX4.columns[[1,2,3,4,13,14]], axis='columns')

In [173]:
X1=X1.values
X2=X2.values
X3=X3.values
X4=X4.values

NX1=NX1.values
NX2=NX2.values
NX3=NX3.values
NX4=NX4.values

print(X1.shape, X2.shape, X3.shape, X4.shape)
print(NX1.shape, NX2.shape, NX3.shape, NX4.shape)

(5900, 10) (5900, 10) (5900, 9) (5900, 9)
(5600, 10) (5600, 10) (5600, 9) (5600, 9)


In [255]:
#X=X1
X=X2
#X=X3
#X=X4

#NX=NX1
NX=NX2
#NX=NX3
#NX=NX4

print(X.shape, NX.shape)

(5900, 10) (5600, 10)


In [256]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X[:]=scaler.fit_transform(X[:])
NX[:]=scaler.fit_transform(NX[:])

In [257]:
X = X.reshape(590,10, 10)
NX = NX.reshape(560,10, 10)
print(X[0].shape)

(10, 10)


In [258]:
label_X= np.zeros((590,1))
label_NX=np.ones((560,1))
label=np.r_[label_X,label_NX]
print(label.shape)

(1150, 1)


In [259]:
data=np.r_[X,NX]
print(data.shape)

(1150, 10, 10)


In [260]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.4, random_state=115)

In [261]:
from tensorflow.keras import utils as np_utils
#one-hot encoding
y_train      = np_utils.to_categorical(y_train)
y_test       = np_utils.to_categorical(y_test)

print(y_train.shape, y_test.shape)

(690, 2) (460, 2)


In [262]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)

print(X_train.shape, X_test.shape)

(690, 10, 10, 1) (460, 10, 10, 1)


In [263]:
from keras.models import Sequential, Model 
from keras.utils import np_utils
from keras.layers import  BatchNormalization,Dense, Conv2D, Convolution2D, MaxPooling2D, Dropout, Flatten, TimeDistributed, InputLayer, LSTM
from keras.layers import Input, Reshape, Activation, add, Add
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

In [264]:
def basic_cnn():
    model = Sequential()

    model.add(Conv2D(input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]), filters = 50, kernel_size = (2,2), strides = (1,1), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 50, kernel_size = (2,2), strides = (1,1), padding = 'same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(filters = 50, kernel_size = (2,2), strides = (1,1), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 50, kernel_size = (2,2), strides = (1,1), padding = 'same'))
    model.add(Activation('relu'))

    # prior layer should be flattend to be connected to dense layers
    model.add(Flatten())
    
    
    # final layer with 10 neurons to classify the instances
    model.add(Dense(50, activation = 'relu'))
    
    # final layer with 10 neurons to classify the instances
    model.add(Dense(2, activation = 'softmax'))

    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

model = basic_cnn()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 10, 10, 50)        250       
_________________________________________________________________
activation_17 (Activation)   (None, 10, 10, 50)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 50)        10050     
_________________________________________________________________
activation_18 (Activation)   (None, 10, 10, 50)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 5, 5, 50)          10050     
_________________________________________________________________
activation_19 (Activation)   (None, 5, 5, 50)         

In [ ]:
hist=model.fit(X_train, y_train, epochs=300, batch_size=50, validation_data=(X_test,y_test))

Train on 690 samples, validate on 460 samples
Epoch 1/300
690/690 [==============================] - 0s 410us/step - loss: 0.6902 - accuracy: 0.5246 - val_loss: 0.6979 - val_accuracy: 0.4783
Epoch 2/300
690/690 [==============================] - 0s 142us/step - loss: 0.6727 - accuracy: 0.5522 - val_loss: 0.6933 - val_accuracy: 0.4783
Epoch 3/300
690/690 [==============================] - 0s 136us/step - loss: 0.6405 - accuracy: 0.6232 - val_loss: 0.6445 - val_accuracy: 0.6326
Epoch 4/300
690/690 [==============================] - 0s 145us/step - loss: 0.6033 - accuracy: 0.6623 - val_loss: 0.5872 - val_accuracy: 0.6826
Epoch 5/300
690/690 [==============================] - 0s 142us/step - loss: 0.4592 - accuracy: 0.8058 - val_loss: 0.3703 - val_accuracy: 0.8630
Epoch 6/300
690/690 [==============================] - 0s 137us/step - loss: 0.3099 - accuracy: 0.8812 - val_loss: 0.2318 - val_accuracy: 0.9043
Epoch 7/300
690/690 [==============================] - 0s 139us/step - loss: 0.2298 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

results = model.evaluate(X_test, y_test)
print('Test accuracy: ', results[1])